### mcp_server.py에서 정의한 neo4j retriever 가지고 답변 해보기

In [1]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini", 
                 temperature=0.3)

# RAG 서버를 위한 StdIO 서버 파라미터 설정
server_params = StdioServerParameters(
    command="/opt/anaconda3/envs/boaz/bin/python",
    args=["mcp_neo4j_server.py"]
)

# StdIO 클라이언트를 사용하여 RAG 서버와 통신
async with stdio_client(server_params) as (read, write):
    # 클라이언트 세션 생성
    async with ClientSession(read, write) as session:
        # 연결 초기화
        await session.initialize()

        # MCP 도구 로드 (여기서는 retriever 도구)
        tools = await load_mcp_tools(session)
        print(f"서버에서 가져온 툴 : {tools}")

        # 에이전트 생성 및 실행
        agent = create_react_agent(model, tools)

        # 에이전트 응답 스트리밍
        result = await agent.ainvoke({
    "messages": [
        {"role": "system", "content": "You are a helpful assistant for medical patient queries. Based on the retrieved data from the neo4j_retriever, answer the user's question."},
        {"role": "user", "content": 'MATCH (p:Patient)-[:UNDERWENT]->(s:Surgery)-[:HAS_RESULT]->(r:Result) WHERE p.name CONTAINS "환아 A" RETURN s.name AS 수술명, r.description AS 수술후상태'}
    ]
})
        for msg in result['messages']:
            role = msg.__class__.__name__
            if hasattr(msg, "content") and msg.content:
                # content가 있을 때만 출력
                print(f"{role} : {msg.content}")
            elif role == "AIMessage":
                # content가 비어 있고 AIMessage라면 tool_calls 확인
                tool_calls = getattr(msg, "tool_calls", None)
                if tool_calls:
                    print(f"{role} : tool_calls → {tool_calls}")
                else:
                    print(f"{role} : [No content and no tool_calls]")
            else:
                print(f"{role} : [No content]")

  + Exception Group Traceback (most recent call last):
  |   File "/opt/anaconda3/envs/boaz/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3665, in run_code
  |     await eval(code_obj, self.user_global_ns, self.user_ns)
  |   File "/var/folders/f_/n_b08qld7fx9lkzrw7kpc0180000gn/T/ipykernel_48159/2007441988.py", line 20, in <module>
  |     async with stdio_client(server_params) as (read, write):
  |   File "/opt/anaconda3/envs/boaz/lib/python3.11/contextlib.py", line 231, in __aexit__
  |     await self.gen.athrow(typ, value, traceback)
  |   File "/opt/anaconda3/envs/boaz/lib/python3.11/site-packages/mcp/client/stdio/__init__.py", line 178, in stdio_client
  |     async with (
  |   File "/opt/anaconda3/envs/boaz/lib/python3.11/site-packages/anyio/_backends/_asyncio.py", line 772, in __aexit__
  |     raise BaseExceptionGroup(
  | ExceptionGroup: unhandled errors in a TaskGroup (1 sub-exception)
  +-+---------------- 1 ----------------
    | Exception Group Trac

### MultiServer 구현해보기

- 1) Neo4J 
- 2) Chroma VectorDB

In [ ]:
from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from langchain_mcp_adapters.tools import load_mcp_tools
from langgraph.prebuilt import create_react_agent
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_mcp_adapters.client import MultiServerMCPClient
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableConfig
import asyncio

load_dotenv()

model = ChatOpenAI(model="gpt-4o-mini", 
                 temperature=0.3)

# 1. 다중 서버 MCP 클라이언트 생성
client = MultiServerMCPClient(
    {
        "neo4j_retriever": {
            "command": "/opt/anaconda3/envs/boaz/bin/python",
            "args": ["mcp_neo4j_server.py"],
            "transport": "stdio",
        },
        "VectorDB_retriever": {
            "command": "/opt/anaconda3/envs/boaz/bin/python",
            "args": ["mcp_vectordb_server.py"],
            "transport": "stdio",
        },
    }
)

# all_tools = await client.get_tools()
# print(all_tools)


config = RunnableConfig(recursion_limit=30, thread_id=1)
# 3. 모든 MCP 서버에서 툴 수집
async def run_agent(question):
    tools = await client.get_tools()
    print(f"사용 가능한 툴 목록: {[tool.name for tool in tools]}")

    # 4. LangGraph 기반 ReAct Agent 생성
    agent = create_react_agent(
        model=model,
        tools=tools,
        prompt=(
            """You have access to two tools:
- `neo4j_retriever`: Use this **only** if the user's question involves **specific patient records**, such as querying a patient's past surgeries, lab results, or procedures recorded in a graph database.
- `VectorDB_retriever`: Use this for **general medical knowledge**, including definitions, typical ranges, procedures, or any explanation not tied to a specific patient's data.

The user may ask questions that involve both types of information. Choose one or both tools appropriately, but do not assume something is a patient query unless it clearly refers to a known patient."""),
        checkpointer=MemorySaver(),
        
    )

    # 6. 에이전트 실행
    result = await agent.ainvoke({
        "messages": [
            {"role": "user", "content": question}
        ]
    },
    config=config)

    # 7. 응답 출력
    for msg in result['messages']:
        role = msg.__class__.__name__
        if hasattr(msg, "content") and msg.content:
            # content가 있을 때만 출력
            print(f"{role} : {msg.content}")
        elif role == "AIMessage":
            # content가 비어 있고 AIMessage라면 tool_calls 확인
            tool_calls = getattr(msg, "tool_calls", None)
            if tool_calls:
                print(f"{role} : tool_calls → {tool_calls}")
            else:
                print(f"{role} : [No content and no tool_calls]")
        else:
            print(f"{role} : [No content]")

# user_input = "수술 중 칼슘 수치가 위험한 수준으로 떨어진 시점은 언제인가요?"
user_input = "Kasabach-Merrritt Syndrome 환자를 수술한 케이스가 있다면 어떻게 수술했는지 알려줘"
await run_agent(user_input)

사용 가능한 툴 목록: ['neo4j_retriever', 'VectorDB_retriever']
HumanMessage : Kasabach-Merrritt Syndrome 환자를 수술한 케이스가 있다면 어떻게 수술했는지 알려줘
AIMessage : tool_calls → [{'name': 'neo4j_retriever', 'args': {'query': 'Kasabach-Merritt Syndrome 환자 수술 케이스'}, 'id': 'call_1J2o20OeIbQjKqnG6UM8vRqs', 'type': 'tool_call'}]
ToolMessage : Error: ToolException('Error executing tool neo4j_retriever: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input \'Kasabach\': expected \'FOREACH\', \'ALTER\', \'ORDER BY\', \'CALL\', \'USING PERIODIC COMMIT\', \'CREATE\', \'LOAD CSV\', \'START DATABASE\', \'STOP DATABASE\', \'DEALLOCATE\', \'DELETE\', \'DENY\', \'DETACH\', \'DROP\', \'DRYRUN\', \'FINISH\', \'GRANT\', \'INSERT\', \'LIMIT\', \'MATCH\', \'MERGE\', \'NODETACH\', \'OFFSET\', \'OPTIONAL\', \'REALLOCATE\', \'REMOVE\', \'RENAME\', \'RETURN\', \'REVOKE\', \'ENABLE SERVER\', \'SET\', \'SHOW\', \'SKIP\', \'TERMINATE\', \'UNWIND\', \'USE\' or \'WITH\' (line 1, column 1 (offset: 0))\n"Kasabach-Merritt Syn